In [1]:
import copy
import numpy as np
import pandas as pd

In [2]:
# 读取原始数据
data_pre = pd.read_excel('./data.xlsx')

In [3]:
data_pre

,风场名,A（高频率穿越能力）,B（低频率穿越能力）,C（低压穿越能力）,D（并网点电压偏差越限次数 ）,E（SVC/SVG响应性能指标）,F（有功控制能力）,G（功率因素越限）
0,风场1,0.7430,0.8267,0.8324,12,0.8637,0.0743,0.0409
1,风场2,0.7567,0.8033,0.8736,10,0.8538,0.0665,0.0716
2,风场3,0.8104,0.7667,0.8539,16,0.9038,0.0881,0.0657


In [4]:
# 指标正向化 D F G 
data = copy.deepcopy(data_pre)
k = 0.1
column_index = np.array([4, 6, 7])
for i in column_index:
    data.iloc[:, i] = 1/(data.iloc[:, i] + np.max(data.iloc[:, i]) + k)

In [5]:
data

,风场名,A（高频率穿越能力）,B（低频率穿越能力）,C（低压穿越能力）,D（并网点电压偏差越限次数 ）,E（SVC/SVG响应性能指标）,F（有功控制能力）,G（功率因素越限）
0,风场1,0.7430,0.8267,0.8324,0.035587,0.8637,3.810976,4.705882
1,风场2,0.7567,0.8033,0.8736,0.038314,0.8538,3.927730,4.111842
2,风场3,0.8104,0.7667,0.8539,0.031153,0.9038,3.620565,4.214075


In [6]:
# 数据标准化
labels = data.keys()[1:]

In [7]:
labels

Index(['A（高频率穿越能力）', 'B（低频率穿越能力）', 'C（低压穿越能力）', 'D（并网点电压偏差越限次数 ）',
       'E（SVC/SVG响应性能指标）', 'F（有功控制能力）', 'G（功率因素越限）'],
      dtype='object')

In [8]:
names = data['风场名']

In [9]:
names

0    风场1
1    风场2
2    风场3
Name: 风场名, dtype: object

In [10]:
values = data[labels].values

In [11]:
values

array([[0.743     , 0.8267    , 0.8324    , 0.03558719, 0.8637    ,
        3.81097561, 4.70588235],
       [0.7567    , 0.8033    , 0.8736    , 0.03831418, 0.8538    ,
        3.92772977, 4.11184211],
       [0.8104    , 0.7667    , 0.8539    , 0.03115265, 0.9038    ,
        3.62056481, 4.21407501]])

In [12]:
for i in range(len(labels)):
    values[:, i] = values[:, i]/np.sqrt(np.sum(values[:, i] ** 2))    

In [13]:
# 计算每列的均值
values_mean = np.mean(values, axis=0)
# 计算每列的标准差
values_std = np.std(values, axis=0)
# 计算变异系数
v_values = values_std/values_mean

In [14]:
v_values

array([0.03777284, 0.03090712, 0.01971778, 0.08427819, 0.0247411 ,
       0.0334341 , 0.05969646])

In [15]:
# 计算权重
w = v_values/np.sum(v_values)

In [16]:
w

array([0.1300057 , 0.10637542, 0.06786419, 0.29006673, 0.08515334,
       0.11507271, 0.20546192])

In [17]:
# 计算得分
score = w @ values.T

In [18]:
score

array([0.587392  , 0.58711278, 0.55471112])

In [19]:
Score = 100 * score / np.max(score)

In [20]:
Score

array([100.        ,  99.95246424,  94.43627314])

In [21]:
for i in range(len(names)):
    print("{}的评分为{}".format(names[i], Score[i]))

风场1的评分为100.0
风场2的评分为99.9524642392077
风场3的评分为94.436273143521
